In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!git clone https://github.com/jacobtordjman/SmartAssessor.git


Cloning into 'SmartAssessor'...
remote: Enumerating objects: 359, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 359 (delta 12), reused 28 (delta 10), pack-reused 320 (from 1)
Receiving objects: 100% (359/359), 75.25 MiB | 27.85 MiB/s, done.
Resolving deltas: 100% (152/152), done.


In [ ]:
import os, pathlib, shutil

MODEL_DIR = "/content/drive/MyDrive/models/WizardMath-7B-V1.1"   # base model goes here
ADAPTER_DIR = "/content/drive/MyDrive/SmartAssessor/wizardmath-lora-checkpoints"  # LoRA adapters

os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(ADAPTER_DIR, exist_ok=True)

print("MODEL_DIR =", MODEL_DIR)
print("ADAPTER_DIR =", ADAPTER_DIR)




MODEL_DIR = /content/drive/MyDrive/models/WizardMath-7B-V1.1
ADAPTER_DIR = /content/drive/MyDrive/SmartAssessor/wizardmath-lora-checkpoints


In [ ]:
# Optional: copy adapters from the repo into Drive (only if not already there)
repo_adapters = pathlib.Path("/content/SmartAssessor/wizardmath-lora-checkpoints")
if repo_adapters.exists():
    if not pathlib.Path(ADAPTER_DIR, "adapter_model.safetensors").exists():
        print("Copying adapters to Drive...")
        shutil.copytree(repo_adapters, ADAPTER_DIR, dirs_exist_ok=True)
else:
    print("Repo adapters not found at", repo_adapters)

In [ ]:
!pip -q install transformers peft accelerate bitsandbytes

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if device == "cuda" else torch.float32
device_map = "auto" if device == "cuda" else None

# Tokenizer from Drive (offline)
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR, use_fast=True, local_files_only=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# 8-bit quantization on GPU; CPU loads standard weights
bnb_config = BitsAndBytesConfig(load_in_8bit=True, llm_int8_enable_fp32_cpu_offload=True) if device=="cuda" else None

# Base model from Drive (offline)
if bnb_config is not None:
    base = AutoModelForCausalLM.from_pretrained(
        MODEL_DIR,
        quantization_config=bnb_config,
        device_map=device_map,
        torch_dtype=torch_dtype,
        trust_remote_code=True
    )
else:
    base = AutoModelForCausalLM.from_pretrained(
        MODEL_DIR,
        torch_dtype=torch_dtype,
        trust_remote_code=True,
        low_cpu_mem_usage=True
    )

# Attach LoRA adapters from Drive
model = PeftModel.from_pretrained(base, ADAPTER_DIR, torch_dtype=torch_dtype)
if device_map is None:
    model = model.to(device)
model.eval()



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 10.2 MB/s eta 0:00:00


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear8bitLt(
                (base_layer): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k

In [ ]:
##!pip -q install huggingface_hub
##
##from huggingface_hub import snapshot_download
##from pathlib import Path
##
### Faster downloads (optional)
### !pip -q install hf-transfer
### import os; os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
##
##if not (Path(MODEL_DIR) / "config.json").exists():
##    print("Downloading model to", MODEL_DIR)
##    snapshot_download(
##        repo_id="WizardLMTeam/WizardMath-7B-V1.1",
##        local_dir=MODEL_DIR,
##        local_dir_use_symlinks=False
##    )
##else:
##    print("Model already present at", MODEL_DIR)
##
### After it’s downloaded once, you can go fully offline:
##import os
##os.environ["HF_HUB_OFFLINE"] = "1"
##os.environ["TRANSFORMERS_OFFLINE"] = "1"
##

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:982: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/948 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

pytorch_model.bin.index.json: 0.00B [00:00, ?B/s]

In [ ]:
# One complete test: full output, no parsing
import torch

# Use the already loaded 'tokenizer' and 'model'
device = next(model.parameters()).device

question = "Compute the determinant of the 2x2 matrix [[-3,2],[1,4]]."

prompt = (
    "You are an expert in linear algebra.\n"
    "Solve the problem step by step, then end with:\n"
    "Final Answer: <number>\n\n"
    f"Problem: {question}\n"
    "Solution:\n"
)

print("=== Prompt ===")
print(prompt)

inputs = tokenizer(prompt, return_tensors="pt")
inputs = {k: v.to(device) for k, v in inputs.items()}

with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=160,     # increase if the answer is cut off
        do_sample=False,        # greedy (no temperature warning)
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        use_cache=True
    )

text = tokenizer.decode(output[0], skip_special_tokens=True)

print("\n=== Model Output ===")
print(text)

# Optional: show expected value for manual check
print("\n(Expected: 10)")


=== Prompt ===
You are an expert in linear algebra.
Solve the problem step by step, then end with:
Final Answer: <number>

Problem: Compute the determinant of the 2x2 matrix [[-3,2],[1,4]].
Solution:


=== Model Output ===
You are an expert in linear algebra.
Solve the problem step by step, then end with:
Final Answer: <number>

Problem: Compute the determinant of the 2x2 matrix [[-3,2],[1,4]].
Solution:
The determinant of a 2x2 matrix [[a, b], [c, d]] is given by ad - bc.
Here, a = -3, b = 2, c = 1, d = 4.
So, det = (-3)*(4) - (2)*(1) = -12 - 2 = -14.

(Expected: 10)


In [ ]:
# Start Cloudflare Tunnel in background and print the public URL
import os, subprocess, time, re
import shlex
if not os.path.exists('cloudflared'):
    !wget -q -O cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
    !chmod +x cloudflared
cf_log = open('cloudflared.log','ab')
cf = subprocess.Popen(['./cloudflared','tunnel','--url','http://localhost:8000','--logfile','cloudflared.log'], stdout=cf_log, stderr=subprocess.STDOUT)
with open('cloudflared.pid','w') as f: f.write(str(cf.pid))
print('cloudflared PID =', cf.pid)
# Wait up to ~30s for the URL to appear in the log
url = None
for _ in range(30):
    time.sleep(1)
    try:
        logtxt = open('cloudflared.log','r', errors='ignore').read()
        m = re.search(r'https://[\w.-]+\.trycloudflare\.com', logtxt)
        if m:
            url = m.group(0)
            break
    except Exception:
        pass
print('Public URL:', url or '(still starting...)')
if url:
    print('Set this in your browser console on GitHub Pages:')
    print(f"localStorage.setItem('API_BASE', '{url}')")

cloudflared PID = 3218
Public URL: https://swedish-becomes-continues-polished.trycloudflare.com
Set this in your browser console on GitHub Pages:
localStorage.setItem('API_BASE', 'https://swedish-becomes-continues-polished.trycloudflare.com')
